In [ ]:
from nba_api.stats.static import teams
import pandas as pd
import numpy as np
import sqlite3
import re
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)
conn = sqlite3.connect("nba.db")
cursor = conn.cursor()

In [ ]:
nba_teams = teams.get_teams()
nba_teams_id = {}
for x in nba_teams:
    name = x['full_name']
    nba_teams_id[f'{name} Id'] = x['id']
ids = nba_teams_id.values()


In [ ]:
from nba_api.stats.endpoints import leaguegamefinder
szn_20_21 = leaguegamefinder.LeagueGameFinder(date_from_nullable="2020-12-22",
                                                date_to_nullable="2021-7-22", 
                                                season_type_nullable="Regular Season",
                                                league_id_nullable='00')
szn_21_22 = leaguegamefinder.LeagueGameFinder(date_from_nullable="2021-10-19",
                                              date_to_nullable="2022-7-19",
                                              season_type_nullable="Regular Season",
                                              league_id_nullable="00")
szn_22_23 = leaguegamefinder.LeagueGameFinder(date_from_nullable="2022-10-18",
                                              date_to_nullable="2023-7-12",
                                              season_type_nullable="Regular Season",
                                              league_id_nullable="00")
szn_23_24 = leaguegamefinder.LeagueGameFinder(date_from_nullable="2023-10-24",
                                              date_to_nullable="2024-4-14",
                                              season_type_nullable="Regular Season",
                                              league_id_nullable="00")
szn_24_25 = leaguegamefinder.LeagueGameFinder(date_from_nullable="2024-10-22",
                                                date_to_nullable="2025-4-13", 
                                                season_type_nullable="Regular Season",
                                                league_id_nullable='00')
szn_20_21 = szn_20_21.get_data_frames()[0]
szn_20_21 = pd.DataFrame(szn_20_21)
gameid = szn_20_21['GAME_ID'].unique()
szn_21_22 = szn_21_22.get_data_frames()[0]
szn_21_22 = pd.DataFrame(szn_21_22)
szn_22_23 = szn_22_23.get_data_frames()[0]
szn_22_23 = pd.DataFrame(szn_22_23)
szn_23_24 = szn_23_24.get_data_frames()[0]
szn_23_24 = pd.DataFrame(szn_23_24)
szn_24_25 = szn_24_25.get_data_frames()[0]
szn_24_25 = pd.DataFrame(szn_24_25)

In [ ]:
szn_20_21.columns

In [ ]:
lst=[szn_20_21, szn_21_22, szn_22_23, szn_23_24, szn_24_25]
for x in lst:
    x["TotalFGMade"] = x.groupby('GAME_ID')['FGM'].transform('sum')
    x["TotalFGAtt"] = x.groupby('GAME_ID')['FGA'].transform('sum')
    x['Total3PM'] = x.groupby('GAME_ID')['FG3M'].transform('sum')
    x["TotalTOV"] = x.groupby('GAME_ID')['TOV'].transform('sum')
    x["TotalFTA"] = x.groupby('GAME_ID')['FTA'].transform('sum')
    x['TotalOREB'] = x.groupby('GAME_ID')['OREB'].transform('sum')
    x['OppOREB'] = x['TotalOREB'] - szn_20_21['OREB']
    x['TDREB'] = x.groupby('GAME_ID')['DREB'].transform('sum')


In [ ]:
##Offensive Team Statistics
def EFG(x):
    num = float(x['FGM']) + 0.5 * float(x['FG3M'])
    denom = float(x['FGA'])
    return round(num/denom, 3)
def TOVP(x):
    denom = float(x['FGA']) + 0.44 * x['FTA'] + float(x['TOV'])
    return round(float(x['TOV']) / denom, 4)
def OREBP(x):
    x['OPPDREB'] = x['TDREB'] - x['DREB']
    return float(x['OREB']) / (float(x['OREB'] + x['OPPDREB']))
def FTRATE(x):
    return float(x['FTA']) / float(x['FGA'])

In [ ]:
##Defensive Team Statistics
def OppEFG(x):
    num = float(x["TotalFGMade"] - x["FGM"]) + 0.5 * float(x["Total3PM"] - x["FG3M"])
    denom = float(x['TotalFGAtt'] - x["FGA"])
    return round(num/denom, 3)
def OppTOVP(x):
    num = float(x['TotalTOV'] - x['TOV'])
    denom = float(x["TotalFGAtt"] - x["FGA"]) + 0.44 * float(x["TotalFTA"] - x["FTA"]) + num
    return round(num/denom, 3)
def DREBP(x):
    return float(x['DREB']) / float(x['DREB'] + x['OppOREB'])
def OppFTRATE(x):
    return float(x['TotalFTA'] - x['FTA']) / float(x['TotalFGAtt'] - x['FGA'])
def away(x):
    return x["MATCHUP"][:4]
def home(x):
    return x["MATCHUP"][-4:]

In [ ]:
for x in lst:
    x['EFG'] = x.apply(EFG, axis=1)
    x['TOVP'] = x.apply(TOVP, axis=1)
    x['OREBP'] = x.apply(OREBP, axis=1)
    x['FTRATE'] = x.apply(FTRATE, axis=1)
    x['OppEFG'] = x.apply(OppEFG, axis=1)
    x['OppTOVP'] = x.apply(OppTOVP, axis=1)
    x['DREBP'] = x.apply(DREBP, axis=1)
    x['OppFTRATE'] = x.apply(OppFTRATE, axis=1)
    x["AWAY"] = x.apply(away, axis=1)
    x["HOME"] = x.apply(home, axis=1)

In [ ]:
szn_20_21.drop(columns=['TDREB', 'OppOREB', 'TotalOREB', 'TotalFTA', 'TotalTOV','Total3PM', 'TotalFGAtt', 'TotalFGMade'])
szn_21_22.drop(columns=['TDREB', 'OppOREB', 'TotalOREB', 'TotalFTA', 'TotalTOV','Total3PM', 'TotalFGAtt', 'TotalFGMade'])
szn_22_23.drop(columns=['TDREB', 'OppOREB', 'TotalOREB', 'TotalFTA', 'TotalTOV','Total3PM', 'TotalFGAtt', 'TotalFGMade'])
szn_23_24.drop(columns=['TDREB', 'OppOREB', 'TotalOREB', 'TotalFTA', 'TotalTOV','Total3PM', 'TotalFGAtt', 'TotalFGMade'])
szn_24_25.drop(columns=['TDREB', 'OppOREB', 'TotalOREB', 'TotalFTA', 'TotalTOV','Total3PM', 'TotalFGAtt', 'TotalFGMade'])

In [ ]:
for x in lst:
    x["EWMA PTS"] = x["PTS"].iloc[7:].shift(1).ewm(span=7).mean()
    x["EWMA FGA"] = x["FGA"].iloc[7:].shift(1).ewm(span=7).mean()
    x["EWMA FG_PCT"] = x["FG_PCT"].iloc[7:].shift(1).ewm(span=7).mean()
    x["EWMA FG3M"] = x["FG3M"].iloc[7:].shift(1).ewm(span=7).mean()
    x["EWMA FG3A"] = x["FG3A"].iloc[7:].shift(1).ewm(span=7).mean()
    x["EWMA FG3_PCT"] = x["FG3_PCT"].iloc[7:].shift(1).ewm(span=7).mean()
    x["EWMA FTM"] = x["FTM"].iloc[7:].shift(1).ewm(span=7).mean()
    x["EWMA FTA"] = x["FTA"].iloc[7:].shift(1).ewm(span=7).mean()
    x["EWMA FT_PCT"] = x["FT_PCT"].iloc[7:].shift(1).ewm(span=7).mean()
    x["EWMA OREB"] = x["OREB"].iloc[7:].shift(1).ewm(span=7).mean()
    x["EWMA DREB"] = x["DREB"].iloc[7:].shift(1).ewm(span=7).mean()
    x["EWMA AST"] = x["AST"].iloc[7:].shift(1).ewm(span=7).mean()
    x["EWMA STL"] = x["STL"].iloc[7:].shift(1).ewm(span=7).mean()
    x["EWMA TOV"] = x["TOV"].iloc[7:].shift(1).ewm(span=7).mean()
    x["EWMA PF"] = x["PF"].iloc[7:].shift(1).ewm(span=7).mean()
    x["EWMA PLUS_MINUS"] = x["STL"].iloc[7:].shift(1).ewm(span=7).mean()
    x["EWMA EFG"] = x['EFG'].iloc[7:].shift(1).ewm(span=7).mean()
    x["EWMA TOVP"] = x['TOVP'].iloc[7:].shift(1).ewm(span=7).mean()
    x["EWMA OREBP"] = x['OREBP'].iloc[7:].shift(1).ewm(span=7).mean()
    x["EWMA FTRATE"] = x['FTRATE'].iloc[7:].shift(1).ewm(span=7).mean()
    x["EWMA OppEFG"] = x['OppEFG'].iloc[7:].shift(1).ewm(span=7).mean()
    x["EWMA OppTOVP"] = x['OppTOVP'].iloc[7:].shift(1).ewm(span=7).mean()
    x["EWMA DREBP"] = x['DREBP'].iloc[7:].shift(1).ewm(span=7).mean()
    x["EWMA OppFTRATE"] = x['EFG'].iloc[7:].shift(1).ewm(span=7).mean()
    

In [ ]:
szn_20_21.to_sql("2020-2021 Team Data",conn,if_exists="replace",index=False)
szn_21_22.to_sql("2021-2022 Team Data",conn,if_exists="replace",index=False)
szn_22_23.to_sql("2022-2023 Team Data",conn,if_exists="replace",index=False)
szn_23_24.to_sql("2023-2024 Team Data",conn,if_exists="replace",index=False)
szn_24_25.to_sql("2024-2025 Team Data",conn,if_exists="replace",index=False)
lst=[szn_20_21, szn_21_22, szn_22_23, szn_23_24, szn_24_25]

In [ ]:
szn_20_21